In [15]:
import requests
import pandas as pd
import numpy as np
import bs4
import time
from datetime import datetime,  timedelta
from config import Config
from alternative_assets import alternative_assets_tickers

In [16]:
# TODO: Make this function simpler
def get_top_ten_stock_raw_sentiment_info(page_number: int, FINNHUB_API_KEY: str) -> list:
    """Get top ten stock info from the first page of the API.

    param: page_number
    type: int

    param: Finnhub API key
    type: str

    return: top_ten_stock_info
    rtype: list"""
    FILTER = 'all-stocks'
    data = requests.get(f'https://apewisdom.io/api/v1.0/filter/{FILTER}/page/{page_number}').json()
    data = data['results']
    top_ten_stock_info_on_first_page = []
    common_stock_count = 0
    data_index = 0
    while common_stock_count < 10:
        ticker = data[data_index]['ticker']
        # ticker_info = requests.get(f"https://finnhub.io/api/v1/search?q={ticker}&token={FINNHUB_API_KEY}").json()
        # ticker_type = ticker_info['result'][0]['type']
        if ticker not in alternative_assets_tickers:
            common_stock_count += 1
            top_ten_stock_info_on_first_page.append(data[data_index])
        data_index += 1
    return top_ten_stock_info_on_first_page

In [17]:
def get_top_ten_stock_ticker(top_ten_stock_info_on_first_page: list) -> list:
    """Get top ten stock ticker from the first page of the API.

    param: page_number
    type: int

    param: api_data
    type: dict

    return: top_ten_stock_ticker
    rtype: list"""
    top_ten_stock_ticker = []
    for stock in top_ten_stock_info_on_first_page:
        top_ten_stock_ticker.append(stock['ticker'])
    return top_ten_stock_ticker

In [18]:
def get_top_ten_stock_only_sentiment(top_ten_stock_ticker: list) -> list:
    """Get top ten stock sentiment from the first page of the API.

    param: top_ten_stock_ticker
    prarm type: list

    return: top_ten_stock_sentiment
    rtype: list"""
    top_ten_stock_sentiment = []
    for stock in top_ten_stock_ticker:
        html_content = requests.get(
            f'https://apewisdom.io/stocks/{stock}/').text
        # soup is a BeautifulSoup object for parsing HTML
        soup = bs4.BeautifulSoup(html_content, 'html.parser')
        title_div = soup.find_all('div', class_='tile-title')
        for title in title_div:
            if title.text == 'Sentiment':
                value_div = title.findNext('div', class_='tile-value')
                if value_div:
                    sentiment_value = value_div.text
                    sentiment_value = float(
                        sentiment_value.replace('%', '').strip())
                    top_ten_stock_sentiment.append(sentiment_value)
    return top_ten_stock_sentiment

In [19]:
def get_top_ten_stock_mentioning_users(top_ten_stock_ticker: list) -> list:
    """Get top ten stock mentioning users from the first page of the API.

    param: top_ten_stock_ticker
    prarm type: list

    return: top_ten_stock_mentioning_users
    rtype: list"""
    top_ten_stock_mentioning_users = []
    for stock in top_ten_stock_ticker:
        html_content = requests.get(
            f'https://apewisdom.io/stocks/{stock}/').text
        soup = bs4.BeautifulSoup(html_content, 'html.parser')
        title_div = soup.find_all('div', class_='tile-title')
        for title in title_div:
            if title.text == 'mentioning users':
                value_div = title.findNext('div', class_='tile-value')
                if value_div:
                    mentioning_users = value_div.text
                    mentioning_users = float(
                        mentioning_users.split(' ')[0].replace(',', ''))
                    top_ten_stock_mentioning_users.append(mentioning_users)
    return top_ten_stock_mentioning_users

In [20]:
config = Config()
FINNHUB_API_KEY = config.finnhub_api_key

In [21]:
def get_top_ten_stock_historical_price(stock_list: list, time_delta_hours: int, 
                           resolution_in_mins: int, FINNHUB_API_KEY: str) -> pd.DataFrame:
    """Get historical stock price for a list of stocks.

    param: stock_list
    type: list

    param: time_delta_hours
    type: int

    param: resolution_in_mins
    type: int

    param: FINNHUB_API_KEY
    type: str

    return: stock_price_df
    rtype: pd.DataFrame"""
    stock_price_df = pd.DataFrame()
    for stock in stock_list:
        to_time = int(datetime.now().timestamp())  # current time
        # time_delta_hours hours is being converted to seconds before subtracting.
        from_time = to_time - (time_delta_hours * 60 * 60)
        try:
            data = requests.get(
                f'https://finnhub.io/api/v1/stock/candle?symbol={stock}&resolution={resolution_in_mins}&from={from_time}&to={to_time}&token={FINNHUB_API_KEY}').json()
            stock_price_df[stock] = data['c']
        except Exception as e:
            print(f"For {stock} error is {e}.")
    return stock_price_df

In [22]:
def get_top_ten_stock_current_price(stock_list: list, FINNHUB_API_KEY: str) -> pd.DataFrame:
    """Get current stock price for a list of stocks.

    param: stock_list
    type: list

    param: FINNHUB_API_KEY
    type: str

    return: stock_price_df
    rtype: pd.DataFrame"""
    stock_price_df = pd.DataFrame()
    for stock in stock_list:
        try:
            data = requests.get(
                f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}').json()
            stock_price_df[stock] = data['c'] # c is the current price
        except Exception as e:
            print(f"For stock {stock} the error is {e}")
    return stock_price_df

In [23]:
def get_top_ten_stock_fundamentals(stock_list: list, FINNHUB_API_KEY: str) -> pd.DataFrame:
    """Get fundamentals for a list of stocks.

    param: stock_list
    type: list

    param: FINNHUB_API_KEY
    type: str

    return: fundamentals_df
    rtype: pd.DataFrame"""
    fundamentals_df = pd.DataFrame()
    METRICS = ['beta', 'epsTTM', 'peTTM', 'roeTTM', 'dividendYieldIndicatedAnnual', 'totalDebt/totalEquityQuarterly', 'revenueGrowthTTMYoy']
    stock_fundamentals = {}
    for stock in stock_list:
        try:
            data = requests.get(
                f"https://finnhub.io/api/v1/stock/metric?symbol={stock}&metric=all&token={FINNHUB_API_KEY}").json()
            fundamental_for_stock = {}
            metrics_in_data = data['metric'].keys()
            for metric in METRICS:
                if metric not in metrics_in_data:
                    fundamental_for_stock[metric] = np.nan
                    continue    
                fundamental_for_stock[metric] = data['metric'][metric]
            stock_fundamentals[stock] = fundamental_for_stock
        except Exception as e:
            print(f"For stock {stock} the error is {e}")
    
    fundamentals_df = pd.DataFrame(stock_fundamentals).T
    fundamentals_df = fundamentals_df.reset_index().rename(columns={'index': 'ticker'})
    return fundamentals_df

In [24]:
def create_top_ten_stock_info_df(top_ten_stock_info: dict, top_ten_stock_sentiments: dict, 
                                top_ten_stock_mentioning_users: dict, top_ten_stock_fundamentals: pd.DataFrame, 
                                timestamp: datetime) -> pd.DataFrame:
    """Create a DataFrame for the top ten stock info.

    param: top_ten_stock_info
    type: dict

    param: top_ten_stock_sentiments
    type: dict

    param: top_ten_stock_mentioning_users
    type: dict

    return: top_stock_info_df
    rtype: pd.DataFrame"""
    top_stock_info_df = pd.DataFrame(top_ten_stock_info)
    top_stock_info_df['sentiment'] = top_ten_stock_sentiments
    top_stock_info_df['mentioning_users'] = top_ten_stock_mentioning_users
    top_stock_info_df['timestamp'] = timestamp
    top_stock_info_df = top_stock_info_df[['timestamp', 'rank', 'ticker', 'name', 'mentions',
                                           'mentioning_users', 'upvotes', 'sentiment', 'rank_24h_ago', 'mentions_24h_ago']]
    top_ten_stock_info_df = pd.concat([top_stock_info_df, top_ten_stock_fundamentals], axis=1)
    return top_ten_stock_info_df

In [25]:
def main_2():
    """Main function"""

    # Congigurations
    config = Config()
    FINNHUB_API_KEY = config.finnhub_api_key
    PAGE_NUMBER = 1 # Pagination number of the website from were data in being scrapped
    FILTER = 'all-stocks'
    
    timestamp = datetime.now()
    top_ten_stock_raw_sentiment_info = get_top_ten_stock_raw_sentiment_info(PAGE_NUMBER, FINNHUB_API_KEY) # Get raw reddit sentiment info
    top_ten_stock_ticker = get_top_ten_stock_ticker(top_ten_stock_raw_sentiment_info)
    top_ten_stock_sentiment = get_top_ten_stock_only_sentiment(top_ten_stock_ticker)
    top_ten_stock_mentioning_users = get_top_ten_stock_mentioning_users(top_ten_stock_ticker)
    top_ten_stock_fundamentals = get_top_ten_stock_fundamentals(top_ten_stock_ticker, FINNHUB_API_KEY)
    top_ten_stock_info_df = create_top_ten_stock_info_df(top_ten_stock_raw_sentiment_info, top_ten_stock_sentiment, 
                                                         top_ten_stock_mentioning_users,top_ten_stock_fundamentals ,timestamp)
    
    return top_ten_stock_info_df

In [26]:
top_ten_stock_info_df = main_2()
top_ten_stock_info_df

,timestamp,rank,ticker,name,mentions,mentioning_users,upvotes,sentiment,rank_24h_ago,mentions_24h_ago,ticker,beta,epsTTM,peTTM,roeTTM,dividendYieldIndicatedAnnual,totalDebt/totalEquityQuarterly,revenueGrowthTTMYoy
0,2023-05-25 01:00:02.880265,1,NVDA,NVIDIA,2501,1123.0,19470,55.0,3,229,NVDA,1.770654,1.7415,173.5333,18.66000,0.052138,0.4956,0.22
1,2023-05-25 01:00:02.880265,2,AI,C3 AI,1016,720.0,7559,67.0,2,338,AI,2.707838,-2.4261,NaN,-22.98162,NaN,0.0000,14.60
2,2023-05-25 01:00:02.880265,4,AMD,AMD,312,198.0,1904,63.0,7,88,AMD,1.812139,0.2412,131.9033,2.40000,NaN,0.0451,22.20
3,2023-05-25 01:00:02.880265,6,TSLA,Tesla,108,83.0,404,62.0,5,97,TSLA,1.753238,3.3967,49.9490,27.90000,NaN,0.0557,38.34
4,2023-05-25 01:00:02.880265,7,PLTR,Palantir,98,48.0,210,67.0,4,144,PLTR,1.682084,-0.1246,NaN,-7.15660,NaN,0.0000,20.50
5,2023-05-25 01:00:02.880265,8,AAPL,Apple,89,66.0,363,58.0,8,48,AAPL,1.172734,5.8857,28.6089,165.72000,0.559571,1.7635,-0.24
6,2023-05-25 01:00:02.880265,9,SNOW,Snowflake,82,63.0,256,47.0,186,2,SNOW,1.368825,-2.4985,NaN,-11.09831,NaN,0.0000,69.41
7,2023-05-25 01:00:02.880265,10,AMZN,Amazon,54,42.0,431,73.0,34,16,AMZN,1.434995,0.4125,177.2569,4.33000,NaN,0.5612,9.87
8,2023-05-25 01:00:02.880265,11,DTE,DTE Energy,45,40.0,381,69.0,9,47,DTE,0.656205,5.6576,19.8403,11.75000,3.490289,1.8338,-15.52
9,2023-05-25 01:00:02.880265,12,AM,Antero Midstream,44,20.0,132,50.0,17,27,AM,0.892486,0.6923,15.4259,15.11000,8.411215,1.5313,7.66


In [14]:
float("1,20")

ValueError: could not convert string to float: '1,20'

In [85]:
def get_top_stocks_opening_closing_prices(stock_list: list, FINNHUB_API_KEY: str) -> pd.DataFrame:
    """Get current stock price for a list of stocks.

    param: stock_list
    type: list

    param: FINNHUB_API_KEY
    type: str

    return: stock_price_df
    rtype: pd.DataFrame"""
    stock_prices_list = []
    for stock in stock_list:
        try:
            data = requests.get(
                f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}').json()
            stock_opening_closing_price = {"stock": stock, "opening_price": data['o'],  "closing_price": data['c']}
            stock_prices_list.append(stock_opening_closing_price)
        except Exception as e:
            print(f"For stock {stock} the error is {e}")
    stock_price_df = pd.DataFrame(stock_prices_list)
    return stock_price_df

In [86]:
get_top_stocks_opening_closing_prices(top_ten_stock_tickers, FINNHUB_API_KEY)

,stock,opening_price,closing_price
0,AI,25.46,27.7002
1,NVDA,309.01,313.4000
2,EU,2.19,2.4250
3,META,245.41,251.0600
4,AMD,104.68,107.8800
5,MVIS,3.67,3.9650
6,MU,65.75,65.9550
7,AAPL,173.98,174.4350
8,TA,86.00,86.0000
9,PLTR,11.62,11.6700


## Not Working 

In [49]:
def get_twitter_sentiments(top_stock_tickers: list, FINNHUB_API_KEY: str):
    """Get twitter sentiments for a list of stocks.

    param: top_stock_tickers
    type: list

    param: FINNHUB_API_KEY
    type: str

    return: twitter_sentiments_df
    rtype: pd.DataFrame"""
    twitter_sentiments_df = pd.DataFrame()
    for stock in top_stock_tickers:
        try:
            from_date = datetime.now() - timedelta(days=2)
            data = requests.get(
                f'https://finnhub.io/api/v1/social-sentiment?symbol={stock}&from={from_time}&to={to_time}&token={FINNHUB_API_KEY}').json()
            twitter_sentiments_df[stock] = data['buzz']['articlesInLastWeekSentiment']
        except Exception as e:
            print(f"For stock {stock} the error is {e}")
    return twitter_sentiments_df

18

In [70]:
to_date = datetime.now()
from_date = to_date - timedelta(days=2)
from_date = from_date.strftime('%Y-%m-%d')
to_date = to_date.strftime('%Y-%m-%d')
stock = 'AAPL'

data = requests.get(f"https://finnhub.io/api/v1/stock/social-sentiment?symbol=AAPL&from={from_date}&to={to_date}&token={FINNHUB_API_KEY}")

In [71]:
data.json()

{'reddit': [{'atTime': '2023-05-21 18:00:00',
   'mention': 6,
   'positiveScore': 0,
   'negativeScore': -0.9976276000000001,
   'positiveMention': 0,
   'negativeMention': 5,
   'score': -1},
  {'atTime': '2023-05-21 15:00:00',
   'mention': 2,
   'positiveScore': 0,
   'negativeScore': -0.98991675,
   'positiveMention': 0,
   'negativeMention': 2,
   'score': -1},
  {'atTime': '2023-05-21 14:00:00',
   'mention': 4,
   'positiveScore': 0.9947418,
   'negativeScore': -0.9547098333333334,
   'positiveMention': 1,
   'negativeMention': 3,
   'score': -0.4844389860838324},
  {'atTime': '2023-05-21 04:00:00',
   'mention': 1,
   'positiveScore': 0,
   'negativeScore': -0.9999707,
   'positiveMention': 0,
   'negativeMention': 1,
   'score': -1},
  {'atTime': '2023-05-21 02:00:00',
   'mention': 3,
   'positiveScore': 0.82378495,
   'negativeScore': -0.99326585,
   'positiveMention': 1,
   'negativeMention': 2,
   'score': -0.41374225569919326},
  {'atTime': '2023-05-21 01:00:00',
   'men

In [72]:
temp = data.json()

In [73]:
temp['twitter']

[]